In [ ]:
from utils import *

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import json
import pandas as pd

from ortools.sat.python import cp_model
from ortools.sat.python.cp_model import _SumArray

In [ ]:
def solve(full_score):
    model = cp_model.CpModel()
    eq_dict, sym_dict = addAllVars(turn)

    buildScoreConstraint(model, eq_dict, full_score, True)
    model.Add(_SumArray([sym_dict[e] for e in values['units'].keys()]) < 20)

    solver = cp_model.CpSolver()
    solution_printer = DataFrameSolutionAggregator(sym_dict.values())
    status = solver.SearchForAllSolutions(model, solution_printer)
    return solution_printer

In [ ]:
# superb_ruin
combined_scores = [
    ([515, 515], [390, 390]),
    ([565, 565], [440, 440]),
    ([590, 600], [450, 450]),
    ([855, 635], [700, 460]),
    ([1385, 670], [1230, 460]),
    ([1860, 1215], [1700, 990]),
    ([2230, 1815], [2010, 1530])
]

In [ ]:
from score_utils import *
possible_score = getPossibleScores(combined_scores)
len(possible_score)

In [ ]:
possible_score

In [ ]:
player_scores = [
    [565, 440],
    [590, 450],
    [855, 700],
    [1385, 1230],
    [1860, 1700],
    [2230, 2010]
]

In [ ]:
player_2_scores = [
    [565, 440],
    [600, 450],
    [635, 460],
    [670, 460],
    [1215, 990],
    [1815, 1530]
]

# Equations: 

full score = score(units) + score(tech) + score(city) + score(vision)

raw score =  score(units) + score(tech) + score(city)

'remaining stars + spt + extra stars = stars(units) + stars(tech) + star(building)'

for v in list(dic.values()):
    val = solver.Value(v)
    if val != 0:
        print(f'{v}: {val}')

In [ ]:
scores = { # Bardur
    "start": [515, None, None],
    "t0": [565, 1, 6], # move, pop, pop, upgarde, end turn
    "t1": [600, 11, 14], # move, train, end turn
    "t2": [625, 18, 21], # move (on village), move, end turn
    "t3": [1065, 31], # move, tech (org), capture, pop, pop, upgrade, end turn
    "t4": [1215, 990],
    "t5": [1815, 1530]
}

scores_bis = { # Yadakk
    "start": [615, None, None],
    "t0": [640, 6, 11], # road, move, train, end turn (see ruin? see village?)
    "t1": [680, 14, 18], # move (on village), move, end turn, end turn (see village?)
    "t2": [980, 21, 25], # move (on village), capture (workshop), end turn
    "t3": [670, 460],
    "t4": [1215, 990],
    "t5": [1815, 1530]
}

In [ ]:
scores = {
    "start": [515, 390],
    "t0": [565, 440],
    "t1": [600, 450],
    "t2": [635, 460],
    "t3": [670, 460],
    "t4": [1215, 990],
    "t5": [1815, 1530]
}

Moves

- start turn

- move

- city upgrade

- pop

- road building

- tech purchase

- unit training

- see ruin? see village? end turn?

In [1]:
import re
from ortools.sat.python import cp_model
from SolutionAggregator import SolutionAggregator
from utils import *
from file_utils import *
from constr_utils import buildAllConstraints

import pandas as pd
pd.set_option('display.max_columns', None)

scores = { # Bardur
    "start": [515, None, None],
    "t0": [565, 1, 6], # move, pop, pop, upgarde, end turn
    "t1": [600, 11, 14], # move, train, end turn
    "t2": [625, 18, 21], # move (on village), move, end turn
    "t3": [1065, 31], # move, tech (org), capture, pop, pop, upgrade, end turn
    "t4": [1215, 990],
    "t5": [1815, 1530]
}
stars = True
moves = True
turn_list = ["start", "t0", "t1", "t2"]#, "t3"]#, "t4", "t5"]
for turn in turn_list:
    buildDictionnaries(turn, stars, moves)
    full_score = scores[turn][0]
    
    buildAllConstraints(turn, stars, moves)

    setFullScore(turn, full_score)
    if scores[turn][2] is not None:
        deltaMoves = scores[turn][2] - scores[turn][1] - 1 # end turn action
        setDeltaMoves(turn, deltaMoves)
    
    setTribe(turn, "Bardur")

    addInitialState(turn, stars, moves)

if True:
    model.Add(dic["t0_revealed"] == 25)
    model.Add(dic["t1_giant"] == 0)
    model.Add(dic["t1_ruin"] == 0)
    model.Add(dic["t2_level-2_count"] == 1)
    model.Add(dic["t2_ruin"] == 0)
    model.Add(dic["t1_revealed"] == 30 or dic["t1_revealed"] == 28 or dic["t1_revealed"] == 25)
    model.Add(dic["t2_move_attack"] == 0)

# addSolution(turn)

# model.Minimize(
#     sum(
#         get(turn, "ruin") +
#         sum(get(turn, "killed_" + u) for u in getValues()['units'] if u != 'giant') 
#         for turn in turn_list))

solver = cp_model.CpSolver()
output_variables = list(dic.values())
solution_printer = SolutionAggregator(output_variables)
#status = solver.Solve(model)
status = solver.SearchForAllSolutions(model, solution_printer)
print('Solve status: %s' % solver.StatusName(status))

Solve status: OPTIMAL


move_move
move_rider_bump
move_training
move_attack
move_knight_chain
move_tech_purchase
move_road_building
move_population
move_city_upgrade
move_city_capture
move_move
move_rider_bump
move_training
move_attack
move_knight_chain
move_tech_purchase
move_road_building
move_population
move_city_upgrade
move_city_capture

In [2]:
x = solution_printer.get_data_frame()
x.shape

(13, 123)

TODO: on capture of a city, it is not possible to train

t2_warrior should not be able to be 3

In [3]:
c = dropColumns(x); print(c.shape); c
#c[[d for d in c.columns if "train" in d]]

(13, 27)


,t1_warrior,t1_population,t1_trained_warrior,t1_animal,t1_stars,t1_star_spending,t1_animal_stars,t1_warrior_stars,t1_unit_stars,t1_pop_stars,t1_move_move,t1_move_training,t1_move_population,t2_warrior,t2_population,t2_revealed,t2_trained_warrior,t2_animal,t2_stars,t2_star_spending,t2_animal_stars,t2_warrior_stars,t2_unit_stars,t2_pop_stars,t2_move_move,t2_move_training,t2_move_population
0,2,2,1,2,3,2,4,2,2,0,2,1,2,3,4,31,2,4,1,6,8,4,2,4,2,2,3
1,2,2,1,2,3,2,4,2,2,0,2,1,2,2,4,33,1,4,3,4,8,2,0,4,2,1,4
2,2,2,1,2,3,2,4,2,2,0,2,1,2,3,3,32,2,3,3,4,6,4,2,2,2,2,3
3,2,2,1,2,3,2,4,2,2,0,2,1,2,3,3,32,2,3,3,4,6,4,2,2,3,2,2
4,2,2,1,2,3,2,4,2,2,0,2,1,2,2,3,34,1,3,5,2,6,2,0,2,3,1,3
5,2,2,1,2,3,2,4,2,2,0,2,1,2,3,2,33,2,2,5,2,4,4,2,0,3,2,2
6,2,2,1,2,3,2,4,2,2,0,2,1,2,3,4,31,2,4,1,6,8,4,2,4,3,2,2
7,2,2,1,2,3,2,4,2,2,0,2,1,2,2,4,33,1,4,3,4,8,2,0,4,3,1,3
8,2,2,1,2,3,2,4,2,2,0,2,1,2,2,4,33,1,4,3,4,8,2,0,4,4,1,2
9,2,2,1,2,3,2,4,2,2,0,2,1,2,2,2,35,1,2,7,0,4,2,0,0,4,1,2


In [31]:
a = pd.DataFrame(calc_entropy(c[col]) for col in c.columns).T

In [33]:
a.columns = c.columns

In [43]:
a

,t1_warrior,t1_population,t1_trained_warrior,t1_animal,t1_stars,t1_star_spending,t1_animal_stars,t1_warrior_stars,t1_unit_stars,t1_pop_stars,t1_move_move,t1_move_training,t1_move_population,t2_warrior,t2_population,t2_revealed,t2_trained_warrior,t2_animal,t2_stars,t2_star_spending,t2_animal_stars,t2_warrior_stars,t2_unit_stars,t2_pop_stars,t2_move_move,t2_move_training,t2_move_population
0,0.619382,0.619382,0.619382,0.619382,0.619382,0.619382,0.619382,0.619382,0.619382,0.619382,0.391244,0.619382,0.773228,0.961237,1.419556,2.045842,0.961237,1.419556,1.66918,1.760495,1.419556,0.961237,0.995727,1.576621,1.526235,0.961237,1.460485


In [47]:
a.columns[a.loc[0].argmax()]

't2_revealed'

In [46]:
c

,t1_warrior,t1_population,t1_trained_warrior,t1_animal,t1_stars,t1_star_spending,t1_animal_stars,t1_warrior_stars,t1_unit_stars,t1_pop_stars,t1_move_move,t1_move_training,t1_move_population,t2_warrior,t2_population,t2_revealed,t2_trained_warrior,t2_animal,t2_stars,t2_star_spending,t2_animal_stars,t2_warrior_stars,t2_unit_stars,t2_pop_stars,t2_move_move,t2_move_training,t2_move_population
0,2,2,1,2,3,2,4,2,2,0,2,1,2,3,4,31,2,4,1,6,8,4,2,4,2,2,3
1,2,2,1,2,3,2,4,2,2,0,2,1,2,2,4,33,1,4,3,4,8,2,0,4,2,1,4
2,2,2,1,2,3,2,4,2,2,0,2,1,2,3,3,32,2,3,3,4,6,4,2,2,2,2,3
3,2,2,1,2,3,2,4,2,2,0,2,1,2,3,3,32,2,3,3,4,6,4,2,2,3,2,2
4,2,2,1,2,3,2,4,2,2,0,2,1,2,2,3,34,1,3,5,2,6,2,0,2,3,1,3
5,2,2,1,2,3,2,4,2,2,0,2,1,2,3,2,33,2,2,5,2,4,4,2,0,3,2,2
6,2,2,1,2,3,2,4,2,2,0,2,1,2,3,4,31,2,4,1,6,8,4,2,4,3,2,2
7,2,2,1,2,3,2,4,2,2,0,2,1,2,2,4,33,1,4,3,4,8,2,0,4,3,1,3
8,2,2,1,2,3,2,4,2,2,0,2,1,2,2,4,33,1,4,3,4,8,2,0,4,4,1,2
9,2,2,1,2,3,2,4,2,2,0,2,1,2,2,2,35,1,2,7,0,4,2,0,0,4,1,2


In [10]:
import pandas as pd
import numpy as np
import math

def calc_entropy(column):
    """
    Calculate entropy given a pandas series, list, or numpy array.
    """
    # Compute the counts of each unique value in the column
    counts = np.bincount(column)
    # Divide by the total column length to get a probability
    probabilities = counts / len(column)
    
    # Initialize the entropy to 0
    entropy = 0
    # Loop through the probabilities, and add each one to the total entropy
    for prob in probabilities:
        if prob > 0:
            # use log from math and set base to 2
            entropy += prob * math.log(prob, 2)
    
    return -entropy

In [48]:
printValueList(x)

start_warrior 1 [1]
start_level-1 1 [1]
start_tier-1 1 [1]
start_claimed 1 [9]
start_revealed 1 [25]
start_hunting 1 [1]
start_Bardur 1 [1]
start_full_score 1 [515]
start_level-1_count 1 [1]
start_pop_max 1 [1]
start_stars 1 [3]
start_spt 1 [2]
start_star_spending 1 [5]
start_tier-1_cost 1 [5]
start_tier-1_stars 1 [5]
start_tier-2_stars 1 [6]
start_tier-3_stars 1 [7]
start_city_spt 1 [2]
start_tech_stars 1 [5]
t0_warrior 1 [1]
t0_level-1 1 [1]
t0_level-2 1 [1]
t0_population 1 [2]
t0_tier-1 1 [1]
t0_claimed 1 [9]
t0_revealed 1 [25]
t0_hunting 1 [1]
t0_Bardur 1 [1]
t0_full_score 1 [565]
t0_claimed_cst 1 [1]
t0_lvl_1_cst 1 [1]
t0_level-2_spt 1 [1]
t0_level-2_count 1 [1]
t0_pop_min 1 [2]
t0_pop_max 1 [4]
t0_animal 1 [2]
t0_stars 1 [1]
t0_spt 1 [4]
t0_star_income 1 [2]
t0_star_spending 1 [4]
t0_animal_stars 1 [4]
t0_tier-1_cost 1 [5]
t0_tier-1_stars 1 [5]
t0_tier-2_stars 1 [6]
t0_tier-3_stars 1 [7]
t0_city_spt 1 [3]
t0_pop_stars 1 [4]
t0_delta_moves 1 [4]
t0_move_move 1 [1]
t0_move_populati

22932357120

In [5]:
cleaned = chooseBestSolution(x, stars, moves)
cleaned.shape

(20, 125)

In [ ]:
cleaned

In [ ]:
dropColumns(cleaned.drop_duplicates())

In [ ]:
printScenario(x)

In [ ]:
variants = printVariants(x)

In [ ]:
variants

In [ ]:
printVariants(cleaned)

In [ ]:
# def prettyPrint(variants):
#     turns = list(set(map(lambda u: u[0].split("_")[0], variants)))
#     for turn_i in turns:
#         turn_string = ("Turn " + turn_i[1:] + ":" if turn_i.startswith("t") else "Start:")
#         print(turn_string)
#         for element, value in map(lambda u: (" ".join(u[0].split("_")[1:]), u[1]), filter(lambda u: u[0].startswith(turn_i), # variants)):
#             print(element + ":", ", ".join(map(lambda u: str(u), value)))
# prettyPrint(variants)

In [ ]:
printValueList(cleaned)

In [ ]:
printValueList(x)

In [ ]:
{
		"name": "water_temple",
		"cost": 20,
		"pop": 1,
		"requires": "acquatism"
	},{
		"name": "forest_temple",
		"cost": 20,
		"pop": 1,
		"requires": "spiritualism"
	},{
		"name": "mountain_temple",
		"cost": 20,
		"pop": 1,
		"requires": "meditation"
	},{
		"name": "temple",
		"cost": 20,
		"pop": 1,
		"requires": "free spirit"
	},{
		"name": "road_conection",
		"cost": 0,
		"pop": 2,
		"requires": "roads"
	},{
		"name": "water_connection",
		"cost": 0,
		"pop": 2,
		"requires": "sailing"
	}